# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    #TODO 1: Add your code here
    df_eng.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_eng['Original Language'] = 'English'
    
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_fr['Original Language'] = 'French'
    
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_sp['Original Language'] = 'Spanish'
    
    
    df = pd.concat([df_eng,df_fr, df_sp], ignore_index=True) 
    
    return df

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",French
14,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",French
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
23,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",Spanish
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
13,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous...",French
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",Spanish
16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",French


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors = "pt", padding=True)

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    translated = outputs
    decoded = [tokenizer.decode(t, skip_special_tokens = True) for t in translated]
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [8]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df.loc[df['Original Language']=='French','Review'].tolist()
fr_reviews_en = translate(fr_reviews, fr_en_model, fr_en_tokenizer)
print(fr_reviews[0])
print(fr_reviews_en[0])

# filter synopsis in French and translate to English
fr_synopsis = df.loc[df['Original Language']=='French','Synopsis'].tolist()
fr_synopsis_en = translate(fr_synopsis, fr_en_model, fr_en_tokenizer)

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df['Original Language']=='Spanish','Review'].tolist()
es_reviews_en = translate(es_reviews, es_en_model, es_en_tokenizer)

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df['Original Language']=='Spanish','Synopsis'].tolist()
es_synopsis_en = translate(es_synopsis, es_en_model, es_en_tokenizer)

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[df['Original Language']=='French','Review'] =fr_reviews_en
df.loc[df['Original Language']=='French','Synopsis'] = fr_synopsis_en

df.loc[df['Original Language']=='Spanish','Review'] = es_reviews_en
df.loc[df['Original Language']=='Spanish','Synopsis'] = es_synopsis_en


/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


"The Land is an absolutely beautiful film with songs that stay in the head for days. The actors are incredible and their alchemy is palpable. The dance scenes are absolutely dazzling and the story is touching and authentic."
"The Land is an absolute beautiful film with songs that stay in the head for days. The actors are incredible and their alchemy is palpable. The dance scenes are absolute dazzling and the story is touching and authentic."


In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Spanish
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",French
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolute charm film that will ma...",French
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers found themselfs...,"""I can't believe I've been time watching this ...",French
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [26]:
# TODO 5: Update the code below
# load sentiment analysis model

model_name = 'juliensimon/reviews-sentiment-analysis'
sentiment_classifier = pipeline('sentiment-analysis', model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """

    output = classifier(text)
    output_class = ["Positive" if i["label"] == "LABEL_1" else "Negative" for i in output]
    
    return output_class

In [27]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column

review_en = df.loc[:,'Review'].tolist()
review_sentiment = analyze_sentiment(review_en, sentiment_classifier)

print(review_en[0])
print(review_sentiment[0])

df["Sentiment"] = review_sentiment

"The Shawshank Redemption is an inspiring tale of hope and friendship, with outstanding performances by Tim Robbins and Morgan Freeman. The story is incredibly moving and will stay with you long after the credits roll."
Positive


In [28]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers found themselfs...,"""I can't believe I've been time watching this ...",French,Negative
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",Spanish,Negative
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English,Negative
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English,Positive
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total wast of time. The jokes a...",French,Negative
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English,Positive
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete surprise. The jokes a...",French,Negative
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish,Positive
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a bore and fairless film that...",Spanish,Negative
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English,Negative


In [30]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)